# Пайплайн № 2
### База знаний: FAQ - и ничего лишнего

### Установка необходимых библиотек

In [ ]:
%%bash

pip install haystack-ai
pip install "datasets>=2.6.1"
pip install "sentence-transformers>=3.0.0"
pip install pymorphy2

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 67.9 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=eb7e036efa22c2d127f73d328faea666a99ada00b38f8db00748e5bb50933ac9
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


### Подготовка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r drive/MyDrive/HacksAI/Russia24/Dataset Dataset/

Эталонная База Знаний FAQ

In [ ]:
import pandas as pd
import json

excel_file_path = 'Dataset/01_База_знаний.xlsx'
df = pd.read_excel(excel_file_path)

df.head()

,Тема,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Что нельзя публиковать на RUTUBE,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
2,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за искус...,Монетизация на RUTUBE зависит в том числе от к...,МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Почему могут отключить монетизацию на видео и ...,"Для каких статусов доступна монетизация, и поч...","Монетизацию на RUTUBE можно подключить, если в...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,Авторское право,Какой контент можно использовать для монетизац...,"То, что вы создали сами: видео, которое вы сня...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


In [ ]:
from datasets import load_dataset
from haystack import Document

docs = [
    Document(content=row['Вопрос из БЗ'], meta={"url": ""})
    for index, row in df.iterrows()
]

### Инициализируем Трансформер для построения эмбеддинга вопросов

In [ ]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.utils import ComponentDevice, Device

device = ComponentDevice.from_str("cuda:0")

# embed_model = "cointegrated/LaBSE-en-ru"
embed_model = "intfloat/e5-large-v2"

doc_embedder = SentenceTransformersDocumentEmbedder(
    model=embed_model,
    device=device
    )

doc_embedder.warm_up()

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

### Запишем вопросы в векторное хранилище

In [ ]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

docs_with_embeddings = doc_embedder.run(docs)
unique_docs = {doc.id: doc for doc in docs_with_embeddings["documents"]}.values()
document_store.write_documents(list(unique_docs))

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

287

### Инициализируем Трансформер для поиска по эмбеддингам вопросов
он должен совпадать с трансформером, на основе которого строились эмбеддинги

In [ ]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(
    model=embed_model,
    device=device
    )

### Инициализируем Retriever

In [ ]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retriever = InMemoryEmbeddingRetriever(document_store, top_k=1)

## Собираем пайплайн

In [ ]:
from haystack import Pipeline

basic_rag_pipeline = Pipeline()
# Add components to your pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)

# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])

Добавим словарь, по которому будем заменять англицизмы на действительно английские слова (т. к. в базе они хранятся именно на английском)

In [ ]:
replace_dict = {
    'шортс': 'shorts',
    "рутуб": "rutube",
    "рутубе": "rutube",
    "рутуба": "rutube",
    "рутубу": "rutube",
    "ютуб": "youtube",
    "ютубе": "youtube",
    "ютуба": "youtube",
    "ютубу": "youtube",
    "смарт": "smart",
    "тв": "tv",
    "самсунг": "samsung",
    "урл": "url",
}

### Интерфейс для получения ответа из RAG пайплайна

In [ ]:
import re
import time
import pymorphy2
import pandas as pd

def get_answer_from_rag(question: str, basic_rag_pipeline, df: pd.DataFrame = df):
    """
    Функция принимает на вход вопрос, приводит его к нижнему регистру, удаляет знаки препинания,
    заменяет слова на основании словаря (если передан), ищет его через RAG pipeline,
    и возвращает соответствующий ответ вместе с классификаторами.

    Параметры:
    - question (str): Вопрос, который передаётся в RAG pipeline.
    - basic_rag_pipeline: RAG pipeline для обработки запроса.
    - replace_dict (dict, optional): Словарь для замены слов в вопросе (например, {'шортс': 'shorts'}).
    - df (pd.DataFrame, optional): DataFrame, содержащий столбцы 'Вопрос из БЗ', 'Ответ из БЗ',
                                    'Классификатор 1 уровня' и 'Классификатор 2 уровня'.

    Возвращает:
    - Tuple[str, str, str]: Ответ из БЗ, классификатор 1 уровня и классификатор 2 уровня.
                             Если ответ не найден, возвращает "Ответ не найден." и пустые строки.
    """
    start_time = time.time()

    morph = pymorphy2.MorphAnalyzer()

    question = question.lower()
    question = re.sub(r'[^\w\s-]', '', question)

    if replace_dict:
        words = question.split()

        replaced_words = []
        for word in words:
            parsed = morph.parse(word)
            if parsed:
                lemma = parsed[0].normal_form
            else:
                lemma = word
            replaced_word = replace_dict.get(lemma, word)
            replaced_words.append(replaced_word)

        question = ' '.join(replaced_words)

    print(f"Processed question: {question}")

    response = basic_rag_pipeline.run({"text_embedder": {"text": question}})

    try:
        target_q = response['retriever']['documents'][0].content
    except (IndexError, KeyError):
        return "Документы не найдены в ответе RAG pipeline.", "", ""

    try:
        target_answer = df.loc[df['Вопрос из БЗ'] == target_q].iloc[0]
        answer_text = target_answer['Ответ из БЗ']
        class_1 = target_answer['Классификатор 1 уровня']
        class_2 = target_answer['Классификатор 2 уровня']

        print(target_answer)

        return answer_text, class_1, class_2
    except IndexError:
        return "Ответ не найден.", "", ""

get_answer_from_rag("Как загрузит?", basic_rag_pipeline)

Processed question: как сменить пароль


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Тема                                                          Общие вопросы
Вопрос из БЗ                                            Как сменить пароль?
Ответ из БЗ               Авторизуйтесь на RUTUBE, перейдите в свой проф...
Классификатор 1 уровня                                 УПРАВЛЕНИЕ АККАУНТОМ
Классификатор 2 уровня                                       Персонализация
Name: 236, dtype: object


('Авторизуйтесь на RUTUBE, перейдите в свой профиль: https://rutube.ru/profile, нажмите «Изменить пароль» и следуйте подсказкам.',
 'УПРАВЛЕНИЕ АККАУНТОМ',
 'Персонализация')

### Асинхронная функция для будущего использования с API

In [ ]:
import asyncio
from pydantic import BaseModel

class Answer(BaseModel):
    answer: str
    class_1: str
    class_2: str

async def get_answer(question: str, basic_rag_pipeline) -> Answer:
    """
    Асинхронная функция принимает вопрос, получает ответ и классы с помощью RAG pipeline
    и возвращает объект AnswerModel.

    Параметры:
    - question (str): Вопрос, который нужно обработать.
    - basic_rag_pipeline: RAG pipeline для обработки запроса.
    - replace_dict (dict, optional): Словарь для замены слов в вопросе.
    - df (pd.DataFrame, optional): DataFrame, содержащий столбцы 'Вопрос из БЗ', 'Ответ из БЗ',
                                    'Классификатор 1 уровня' и 'Классификатор 2 уровня'.

    Возвращает:
    - AnswerModel: Объект с полями answer, class_1 и class_2.
    """
    loop = asyncio.get_event_loop()
    # Запуск синхронной функции в отдельном потоке, чтобы не блокировать event loop
    answer_text, class_1, class_2 = await loop.run_in_executor(
        None,
        get_answer_from_rag,
        question,
        basic_rag_pipeline,
    )

    return Answer(
        answer=answer_text,
        class_1=class_1 if class_1 else "",
        class_2=class_2 if class_2 else "",
    )

In [ ]:
ttest = await get_answer("шортс?", basic_rag_pipeline)
print("-------------")
print(ttest.answer)

Processed question: shorts


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Тема                                                                 Shorts
Вопрос из БЗ                                     Какие требования к Shorts?
Ответ из БЗ               Длительность до 60 секунд. Вертикальная ориент...
Классификатор 1 уровня                                                ВИДЕО
Классификатор 2 уровня                                       Загрузка видео
Name: 108, dtype: object
-------------
Длительность до 60 секунд. Вертикальная ориентация. Высота больше ширины.


### Тест модели

In [ ]:
import pandas as pd

def process_all_questions(input_file, question_col, basic_rag_pipeline, output_file):
    """
    Функция обрабатывает все вопросы из указанного файла, вызывает get_answer_from_rag для каждого,
    и сохраняет результаты в новый CSV файл.

    Параметры:
    - input_file: Путь к CSV или XLSX файлу с вопросами.
    - question_col: Название колонки, содержащей вопросы.
    - basic_rag_pipeline: RAG pipeline для обработки запроса.
    - output_file: Путь для сохранения результирующего CSV файла с вопросами и ответами.

    Возвращает:
    - DataFrame с исходными вопросами и найденными ответами.
    """

    # Определяем расширение файла (CSV или XLSX)
    file_extension = input_file.split('.')[-1].lower()

    # Чтение данных в зависимости от расширения
    if file_extension == 'csv':
        df_questions = pd.read_csv(input_file)
    elif file_extension == 'xlsx':
        df_questions = pd.read_excel(input_file)
    else:
        raise ValueError("Файл должен быть формата CSV или XLSX.")

    # Проверка на наличие колонки с вопросами
    if question_col not in df_questions.columns:
        raise ValueError(f"Указанная колонка '{question_col}' не найдена в файле.")

    # Инициализация списка для хранения результатов
    results = []

    # Проходим по всем вопросам
    for question in df_questions[question_col][:10]:
        # Получаем ответ через функцию get_answer_from_rag
        answer, class_1, class_2 = get_answer_from_rag(question, basic_rag_pipeline)

        # Добавляем результат в список
        results.append({'Вопрос': question, 'Ответ': answer})

    # Создаем DataFrame из результатов
    df_results = pd.DataFrame(results)

    # Сохраняем результат в CSV файл
    # df_results.to_csv(output_file, index=False, encoding='utf-8')

    print(f"Результаты сохранены в файл: {output_file}")
    return df_results

In [ ]:
main_path = "drive/MyDrive/HacksAI/Russia24/"

src_file_path = main_path + "ground_truth_data/pipeline2_g_t_data.csv"
out_file_path = main_path + "generated_data/pipeline2_answers.csv"

my_df = process_all_questions(src_file_path, 'Вопрос', basic_rag_pipeline, out_file_path)
my_df.head()

Processed question: что нельзя публиковать на rutube


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Тема                                       Что нельзя публиковать на RUTUBE
Вопрос из БЗ                              Что нельзя публиковать на RUTUBE?
Ответ из БЗ               Чужой контент без разрешения автора или правоо...
Классификатор 1 уровня                                            МОДЕРАЦИЯ
Классификатор 2 уровня                          Отклонение/блокировка видео
Name: 0, dtype: object
Processed question: почему могут отключить монетизацию из-за авторского права на видео и как этого избежать


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Тема                      Почему могут отключить монетизацию на видео и ...
Вопрос из БЗ              Почему могут отключить монетизацию из-за автор...
Ответ из БЗ               Монетизация может отключиться, если на вашем к...
Классификатор 1 уровня                                          МОНЕТИЗАЦИЯ
Классификатор 2 уровня                   Отключение/подключение монетизации
Name: 1, dtype: object
Processed question: почему могут отключить монетизацию из-за искусственного завышения показателей на видео и как этого избежать 


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Тема                                                        Авторское право
Вопрос из БЗ                        Можно ли цитировать чужие произведения?
Ответ из БЗ               Да, но нужно соблюдать несколько условий: Цити...
Классификатор 1 уровня                                            МОДЕРАЦИЯ
Классификатор 2 уровня                          Отклонение/блокировка видео
Name: 8, dtype: object
Processed question: могу ли я использовать небольшие фрагменты фильмов для создания своих видео


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Тема                                                        Авторское право
Вопрос из БЗ              Могу ли я использовать небольшие фрагменты фил...
Ответ из БЗ               Да, на тех же условиях, что и цитирование. Исп...
Классификатор 1 уровня                                            МОДЕРАЦИЯ
Классификатор 2 уровня                          Отклонение/блокировка видео
Name: 9, dtype: object
Результаты сохранены в файл: drive/MyDrive/HacksAI/Russia24/generated_data/pipeline2_answers.csv


,Вопрос,Ответ
0,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...
1,Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к..."
2,Почему могут отключить монетизацию из-за искус...,Монетизация на RUTUBE зависит в том числе от к...
3,"Для каких статусов доступна монетизация, и поч...","Монетизацию на RUTUBE можно подключить, если в..."
4,Какой контент можно использовать для монетизац...,"То, что вы создали сами: видео, которое вы сня..."
